In [1]:
import pandas as pd
import numpy as np 

In [3]:
gmv = pd.read_excel('C:/Users/Dimon/Downloads/gmv.xlsx', index_col=0, skiprows=2)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Dimon/Downloads/gmv.xlsx'

,Артикул,Наименование,Отдел,ТО,"Маржа, %","Маржа, без НДС","Кол-во, ед.",Объем,Вес
79,10690746,Мешок для строительного мусора 70 л 55x95 см п...,1,1096034.80,0.024984,22831.81,103719.0,46.716490,3941.322
106,12845385,Мешок для строительного мусора 70 л 55x95 см п...,1,936633.93,0.043372,33854.00,61064.0,29.102248,2870.008
502,12902562,Подвес прямой для потолочного профиля 60x27 мм...,1,311013.68,0.097780,25340.56,41077.0,0.776766,1314.464
93,83022208,Перчатки хлопчатобумажные обливные И-8047-И ра...,4,1007303.00,0.218382,183325.22,36142.0,15.607959,1517.964
159,84921534,Кирпич рядовой керамический полнотелый М100 кр...,1,740137.16,0.036822,22711.11,34147.0,66.586650,109270.400
...,...,...,...,...,...,...,...,...,...
33044,13537712,Перчатки хлопчатобумажные обливные размер 9/L ...,4,-336.52,-0.956852,268.69,-22.0,-0.030390,-0.990
33078,81978629,Ламинат Artens «Ясень светлый» 32 класс толщин...,5,-37116.48,0.162596,-5028.95,-23.0,-0.394363,-342.355
33080,10292561,Рамка для розеток и выключателей Legrand Valen...,3,-42205.46,0.165500,-5820.72,-30.0,-0.008103,-2.160
33069,82034158,Шпаклёвка гипсовая финишная Волма Финиш 20 кг,1,-6480.98,-0.769637,4156.51,-36.0,-0.525096,-723.600


объем уникальных артикулов 

In [7]:
units24['Артикул'].value_counts().sum()

33082

ТО суммарное за 8 мес в 24 году

In [10]:
units24['ТО'].sum()

920879847.1099999

Артикул
84698363    1
86797953    1
89343315    1
89343314    1
82979881    1
           ..
87890879    1
13105178    1
81930439    1
83508157    1
82449649    1
Name: count, Length: 33082, dtype: int64